## Sistema: n servidores en serie con retroceso

Este proyecto se enfoca en la creación de una simulación de eventos discretos con el fin de analizar y comprender diversos fenómenos. Nuestro objetivo es aplicar los principios de esta simulación para modelar y experimentar con dichos fenómenos, con el propósito de obtener resultados que guíen nuestras decisiones de manera informada.

Los clientes llegan a un sistema que tiene n servidores, y las llegadas distribuye M. Cada cliente que llega debe ser atendido primero por el servidor 1 y, al completar el servicio en el servidor 1, el cliente pasa al servidor 2.

Cuando un cliente llega, entra en servicio con el servidor 1 si ese servidor está libre, o se une a la cola del servidor 1 en caso contrario. De manera similar, cuando el cliente completa el servicio en el servidor 1, entra en servicio con el servidor 2 si ese servidor está libre, o se une a su cola y asi sucesivamente. Después de ser atendido en el servidor n, el cliente abandona el sistema.

El servidor i con una probabilidad p pude se salta a la cola del servidor j.

Los tiempos de servicio en el servidor i tienen la distribución Gi

### Variables

#### Tiempo

$t$: tiempo general  
$t_A$: tiempo de arribo del proximo cliente  
$t_Di$: tiempo de salido del servidor i  

#### Contadoras

$n_A$: cantidad de arribos  
$n_D$: cantidad de salidas  
$A_i$: arribos al servidor i  
$D_j$: salidas del servidor i  

#### Estado

$Queue_i$: cola de espera del servidor i


### Implementación

En la implementación del proyecto, se utilizan diversas funciones para la generación de números aleatorios con distribuciones específicas, necesarias para la simulación de eventos discretos. Estas funciones se han creado utilizando el módulo random de Python y otras bibliotecas estándar, guiandonos por las distintas funciones de distribución.

In [ ]:
from enum import Enum
import math
import random
from queue import PriorityQueue as pq

def generate_exponential(lambd): 
    return math.log(1 - random.uniform(), math.e) / -lambd

def generate_bernoulli(p):
    return 1 if random.uniform() < p else 0

def generate_poisson(lambd):
    L = math.exp(-lambd)
    k = 0
    p = 1

    while True:
        k += 1
        p *= random.uniform(0, 1)
        if p <= L:
            return k - 1

Se define un enum llamada EventType, el cual representa los distintos tipos de eventos que pueden ocurrir en el sistema simulado. Cada tipo de evento tiene asignado un valor entero único.

ASIGNED: Este evento se refiere a la asignación de un cliente a la cola de un servidor.

ARRIVAL: Representa la llegada de un cliente a un servidor.

FINISH: Indica la finalización o conclusión del servicio al cliente.

In [ ]:
class EventType(Enum):
    ASIGNED = 1
    ARRIVAL = 2
    FINISH = 3

En este proyecto, hemos desarrollado simulaciones para dos tipos de relaciones servidor-cliente, aplicando distribuciones específicas a las variables aleatorias involucradas en cada situación.

Para simular los servidores informáticos, hemos empleado una distribución exponencial con distintos valores de lambda. Estos valores de lambda representan diversas velocidades de procesamiento, que varían según la calidad y la generación de los servidores. Al utilizar distribuciones exponenciales, modelamos el tiempo que tarda cada servidor en completar una tarea, teniendo en cuenta la tasa promedio de llegada de solicitudes.

Por otro lado, en el caso de la simulación de un banco, hemos optado por la distribución de Poisson que permite reflejar procesos más lentos. Esta distribución captura la variabilidad en los tiempos de espera y procesamiento en un entorno bancario, donde los procedimientos suelen ser más detallados y los tiempos de espera pueden ser considerablemente más largos en comparación con las operaciones informáticas estándar.

La selección de distribuciones y valores específicos de lambda o parámetros de distribución se basa en la necesidad de modelar con precisión las características y variaciones de tiempo en cada contexto simulado, lo que nos permite obtener resultados realistas y significativos para la toma de decisiones y la optimización de los sistemas.

### Servidores informáticos

In [ ]:
def simulation(n, lambda_arrival_time, lambda_wait_time):
    """
    Este método simula un sistema de servidores para atender solicitudes de clientes.

    Args:
        n (int): El número de servidores en el sistema.
        lambda_arrival_time (float): La tasa promedio de llegada de solicitudes al sistema.
        lambda_wait_time (float): La tasa promedio de tiempo de espera en el sistema.
        p (float): La probabilidad de que una solicitud sea asignada a un servidor adyacente.

    Returns:
        None
    """
    
    t = 0
    events = pq()
    n_a = 0
    n_d = 0
    a = [{} for _ in range(n)]
    d = [{} for _ in range(n)]
    queue = [[] for _ in range(n)]

    while(t < 864):
        events.put((t + generate_exponential(lambda_arrival_time), 1, 0, 0))
        t, e, c, i = events.get()
        if e == 1:
            if len(queue[i]) == 0:
                events.put((t, 2, c if i !=0 else n_a, i))
            
            queue[i].append(c if i !=0 else n_a)

            if i == 0:
                n_a += 1
        elif e == 2:
            a[i][c] = t
            duration = generate_exponential(lambda_wait_time)
            events.put((t + duration, 3, c, i))
        elif e == 3:
            if len(queue[i]) > 0:
                queue[i].pop(0)
            
            if len(queue[i]) > 0:
                events.put((t, 2, queue[i][0], i))

            d[i][c] = t
            if i == n-1:
                n_d += 1
                continue

            events.put((t, 1, c, i+1))
